In [ ]:
import subprocess
import os
import time
import datetime
from pathlib import Path
import shutil

DELETE_ROOT = r"/home/cenking/VsCode/Delete/Delete_Running"

DELETE_CONFIG = {
    'surf_path': './data/TDFA/TDFA_Macro_pocket_8.0.ply',
    'frag_path': './data/TDFA/TDFA1_Coor.sdf',
    'check_point': './checkpoint/ckpt/delete.pt',
    'outdir': './outputs',
    'suboutdir': 'TDFA1'
}

RUN_DURATION_HOURS = 1

USE_CPU_VERSION = False

WAIT_BETWEEN_RUNS = 10

OUTPUT_NAMING_PATTERN = "{base_name}_Run_{run_number}_{timestamp}"

class DeleteAutoRunner:
    
    def __init__(self, delete_root, config, duration_hours, use_cpu=False):
        self.delete_root = Path(delete_root)
        self.config = config.copy()
        self.duration_hours = duration_hours
        self.use_cpu = use_cpu
        self.run_counter = 1
        self.total_runs = 0
        
        if not self.delete_root.exists():
            raise ValueError(f"Delete root directory does not exist: {self.delete_root}")
        
        self.start_time = datetime.datetime.now()
        self.end_time = self.start_time + datetime.timedelta(hours=duration_hours)
        
        print("=" * 80)
        print("Delete Model Automated Continuous Running System Started (Duration-based)")
        print("=" * 80)
        print(f"Delete root directory: {self.delete_root}")
        print(f"Start time: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Expected end time: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Running duration: {duration_hours} hours")
        print(f"Running mode: {'CPU version' if use_cpu else 'GPU version'}")
        print(f"Base output directory: {config['suboutdir']}")
        print("=" * 80)
    
    def should_start_new_run(self):
        current_time = datetime.datetime.now()
        return current_time < self.end_time
    
    def get_remaining_time(self):
        current_time = datetime.datetime.now()
        remaining = (self.end_time - current_time).total_seconds()
        return max(0, remaining)
    
    def format_time(self, seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours}h {minutes}m {secs}s"
    
    def generate_output_folder_name(self):
        base_name = self.config['suboutdir']
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        date = datetime.datetime.now().strftime("%Y%m%d")
        time_str = datetime.datetime.now().strftime("%H%M%S")
        
        folder_name = OUTPUT_NAMING_PATTERN.format(
            base_name=base_name,
            run_number=self.run_counter,
            timestamp=timestamp,
            date=date,
            time=time_str
        )
        
        return folder_name
    
    def build_command(self, output_folder):
        script_name = "delete_cpu.py" if self.use_cpu else "delete.py"
        
        cmd = [
            "python", "-u", script_name,
            "--surf_path", self.config['surf_path'],
            "--frag_path", self.config['frag_path'],
            "--check_point", self.config['check_point'],
            "--outdir", self.config['outdir'],
            "--suboutdir", output_folder
        ]
        
        return cmd
    
    def run_delete_once(self):
        output_folder = self.generate_output_folder_name()
        
        print(f"\n{'='*80}")
        print(f"Starting run #{self.run_counter}")
        print(f"Output folder: {output_folder}")
        print(f"Start time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"Remaining time: {self.format_time(self.get_remaining_time())}")
        print(f"{'='*80}\n")
        
        cmd = self.build_command(output_folder)
        
        try:
            os.chdir(self.delete_root)
            
            print(f"Executing command: {' '.join(cmd)}\n")
            
            start_time = time.time()
            process = subprocess.Popen(
                cmd,
                stdout=subprocess.PIPE,
                stderr=subprocess.STDOUT,
                text=True,
                bufsize=1
            )
            
            for line in process.stdout:
                print(line, end='')
            
            return_code = process.wait()
            end_time = time.time()
            duration = end_time - start_time
            
            print(f"\n{'='*80}")
            print(f"Run #{self.run_counter} completed")
            print(f"Run duration: {duration/60:.2f} minutes")
            print(f"Return code: {return_code}")
            print(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            output_path = self.delete_root / self.config['outdir'] / output_folder
            if output_path.exists():
                sdf_files = list(output_path.glob("*.sdf"))
                print(f"Files generated: {len(sdf_files)}")
            else:
                print(f"Warning: Output folder not found: {output_path}")
            
            remaining = self.get_remaining_time()
            print(f"Total remaining time: {self.format_time(remaining)}")
            print(f"{'='*80}\n")
            
            self.run_counter += 1
            self.total_runs += 1
            
            return True
            
        except Exception as e:
            print(f"\n{'='*80}")
            print(f"Run #{self.run_counter} encountered an error")
            print(f"Error message: {e}")
            print(f"{'='*80}\n")
            return False
    
    def run_continuous(self):
        print("\nStarting continuous running loop...\n")
        
        try:
            while True:
                if not self.should_start_new_run():
                    print(f"Reached the set running duration ({self.duration_hours} hours)")
                    print(f"No more new runs will be started, preparing to end...")
                    break
                
                print(f"Starting new run (remaining time: {self.format_time(self.get_remaining_time())})")
                success = self.run_delete_once()
                
                if not self.should_start_new_run():
                    print(f"\nSet duration exceeded after run completion")
                    print(f"No more new runs will be started")
                    break
                
                if WAIT_BETWEEN_RUNS > 0:
                    print(f"Waiting {WAIT_BETWEEN_RUNS} seconds before starting next run...\n")
                    time.sleep(WAIT_BETWEEN_RUNS)
                
        except KeyboardInterrupt:
            print(f"\n{'='*80}")
            print("User interrupted the run")
            
        finally:
            actual_duration = (datetime.datetime.now() - self.start_time).total_seconds() / 3600
            
            print(f"\n{'='*80}")
            print("Run ended")
            print(f"Total runs completed: {self.total_runs}")
            print(f"Set running duration: {self.duration_hours} hours")
            print(f"Actual running duration: {actual_duration:.2f} hours")
            print(f"Start time: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            print(f"{'='*80}")

if __name__ == "__main__":
    runner = DeleteAutoRunner(
        delete_root=DELETE_ROOT,
        config=DELETE_CONFIG,
        duration_hours=RUN_DURATION_HOURS,
        use_cpu=USE_CPU_VERSION
    )
    
    runner.run_continuous()

🚀 Delete模型自动化持续运行系统已启动（基于运行时长）
📂 Delete根目录: /home/cenking/VsCode/Delete/Delete_Running
⏰ 开始时间: 2025-12-07 18:26:45
⏰ 预计结束时间: 2025-12-07 19:26:45
⏱️  运行时长: 1 小时
💻 运行模式: GPU版本
🎯 基础输出目录: TDFA1

🎬 开始持续运行循环...

✅ 开始新的运行（剩余时间: 0小时59分59秒）

🔄 开始第 1 次运行
📁 输出文件夹: TDFA1_Run_1_20251207_182645
🕐 开始时间: 2025-12-07 18:26:45
⏳ 剩余时间: 0小时59分59秒

执行命令: python -u delete.py --surf_path ./data/TDFA/TDFA_Macro_pocket_8.0.ply --frag_path ./data/TDFA/TDFA1_Coor.sdf --check_point ./checkpoint/ckpt/delete.pt --outdir ./outputs --suboutdir TDFA1_Run_1_20251207_182645

Num of parameters is 37.06M
Start to generate!
/home/cenking/anaconda3/envs/Delete/lib/python3.9/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4322.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Success: NC(=O)n1c2ccccc2c2ccccc21
Success: FN=Cn1c2ccccc2c2ccccc

'\n📖 使用指南（基于运行时长版本）：\n\n1. 【配置参数】\n   - 修改DELETE_ROOT为您的Delete模型根目录\n   - 修改DELETE_CONFIG中的路径参数\n   - 设置RUN_DURATION_HOURS定义运行时长（单位：小时）\n   \n2. 【运行时长设置】\n   - RUN_DURATION_HOURS = 7: 运行7小时\n   - RUN_DURATION_HOURS = 8: 运行8小时\n   - RUN_DURATION_HOURS = 10: 运行10小时\n   - 可以使用小数，例如 RUN_DURATION_HOURS = 0.5 表示30分钟\n   \n3. 【运行逻辑】\n   - 系统会在设定时长内持续运行Delete模型\n   - 如果最后一次运行开始时还没超时，会等待其完成\n   - 这意味着实际运行时间可能会略超过设定时长\n   \n4. 【运行模式】\n   - USE_CPU_VERSION = True: 使用delete_cpu.py（无GPU）\n   - USE_CPU_VERSION = False: 使用delete.py（有GPU）\n   \n5. 【输出文件夹命名】\n   - 默认格式: 7wmm_Frag2_Run_1_20251112_223045\n   - 可修改OUTPUT_NAMING_PATTERN自定义命名\n   \n6. 【停止运行】\n   - 自动停止: 达到设定时长后自动结束（等待当前运行完成）\n   - 手动停止: 在Jupyter中点击停止按钮\n   \n7. 【监控运行】\n   - 实时显示Delete模型输出\n   - 显示每次运行的时长和文件数\n   - 显示剩余运行时间\n   - 统计总运行次数和实际运行时长\n\n⚠️  注意事项：\n   - 确保Delete环境已激活\n   - 确保所有输入文件路径正确\n   - 建议先手动运行一次测试\n   - 磁盘空间充足（每次运行可能生成大量文件）\n   - 实际运行时间 = 设定时长 + 最后一次运行的时间\n   \n💡 示例：\n   如果设置 RUN_DURATION_HOURS = 7，每次运行需要30分钟\n   - 第1次运行: 0:0

In [ ]:
import subprocess
import os
import time
import datetime
from pathlib import Path
import logging
from tqdm.notebook import tqdm

DELETE_ROOT = r"/home/cenking/VsCode/Delete"

DELETE_CONFIG = {
    'surf_path': './data/workdir/7wmm/7WMM_Macro_pocket_8.0.ply',
    'frag_path': './data/workdir/7wmm/Frag2.sdf',
    'check_point': './checkpoint/ckpt/delete.pt',
    'outdir': './outputs',
    'suboutdir': '7wmm_Frag2'
}

RUN_DURATION_HOURS = 6

USE_CPU_VERSION = True

WAIT_BETWEEN_RUNS = 10

OUTPUT_NAMING_PATTERN = "{base_name}_Run_{run_number}_{timestamp}"

LOG_FILE = "Delete_AutoRun.log"

class DeleteAutoRunnerV2:
    
    def __init__(self, delete_root, config, duration_hours, use_cpu=False, log_file=None):
        self.delete_root = Path(delete_root)
        self.config = config.copy()
        self.duration_hours = duration_hours
        self.use_cpu = use_cpu
        self.run_counter = 1
        self.total_runs = 0
        self.log_file = log_file
        
        if not self.delete_root.exists():
            raise ValueError(f"Delete root directory does not exist: {self.delete_root}")
        
        self._setup_logging()
        
        self.start_time = datetime.datetime.now()
        self.end_time = self.start_time + datetime.timedelta(hours=duration_hours)
        
        self.log("=" * 80)
        self.log("Delete Model Automated Continuous Running System Started (Duration-based)")
        self.log("=" * 80)
        self.log(f"Delete root directory: {self.delete_root}")
        self.log(f"Start time: {self.start_time.strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"Expected end time: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"Running duration: {duration_hours} hours")
        self.log(f"Running mode: {'CPU version' if use_cpu else 'GPU version'}")
        self.log(f"Base output directory: {config['suboutdir']}")
        self.log(f"Log file: {self.log_file}")
        self.log("=" * 80)
        
        print(f"System started, will run for {duration_hours} hours")
        print(f"Detailed logs saved to: {self.log_file}")
        print(f"Expected end time: {self.end_time.strftime('%Y-%m-%d %H:%M:%S')}")
    
    def _setup_logging(self):
        if self.log_file:
            log_path = self.delete_root / self.log_file
        else:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            log_path = self.delete_root / f"Delete_AutoRun_{timestamp}.log"
        
        self.log_file = str(log_path)
        
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler(self.log_file, encoding='utf-8'),
            ]
        )
        self.logger = logging.getLogger(__name__)
    
    def log(self, message):
        self.logger.info(message)
    
    def is_time_remaining(self):
        current_time = datetime.datetime.now()
        return current_time < self.end_time
    
    def get_remaining_time(self):
        current_time = datetime.datetime.now()
        remaining = (self.end_time - current_time).total_seconds()
        return max(0, remaining)
    
    def format_time(self, seconds):
        hours = int(seconds // 3600)
        minutes = int((seconds % 3600) // 60)
        secs = int(seconds % 60)
        return f"{hours}h {minutes}m {secs}s"
    
    def generate_output_folder_name(self):
        base_name = self.config['suboutdir']
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        date = datetime.datetime.now().strftime("%Y%m%d")
        time_str = datetime.datetime.now().strftime("%H%M%S")
        
        folder_name = OUTPUT_NAMING_PATTERN.format(
            base_name=base_name,
            run_number=self.run_counter,
            timestamp=timestamp,
            date=date,
            time=time_str
        )
        
        return folder_name
    
    def build_command(self, output_folder):
        script_name = "delete_cpu.py" if self.use_cpu else "delete.py"
        
        cmd = [
            "python", "-u", script_name,
            "--surf_path", self.config['surf_path'],
            "--frag_path", self.config['frag_path'],
            "--check_point", self.config['check_point'],
            "--outdir", self.config['outdir'],
            "--suboutdir", output_folder
        ]
        
        return cmd
    
    def run_delete_once(self):
        output_folder = self.generate_output_folder_name()
        
        self.log("=" * 80)
        self.log(f"Starting run #{self.run_counter}")
        self.log(f"Output folder: {output_folder}")
        self.log(f"Start time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        self.log(f"Remaining time: {self.format_time(self.get_remaining_time())}")
        self.log("=" * 80)
        
        cmd = self.build_command(output_folder)
        
        try:
            os.chdir(self.delete_root)
            
            self.log(f"Executing command: {' '.join(cmd)}")
            
            start_time = time.time()
            
            with open(self.delete_root / f"run_{self.run_counter}_output.log", 'w') as log_f:
                process = subprocess.Popen(
                    cmd,
                    stdout=log_f,
                    stderr=subprocess.STDOUT,
                    text=True
                )
                
                return_code = process.wait()
            
            end_time = time.time()
            duration = end_time - start_time
            
            self.log("=" * 80)
            self.log(f"Run #{self.run_counter} completed")
            self.log(f"Run duration: {duration/60:.2f} minutes")
            self.log(f"Return code: {return_code}")
            self.log(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            
            output_path = self.delete_root / self.config['outdir'] / output_folder
            if output_path.exists():
                sdf_files = list(output_path.glob("*.sdf"))
                self.log(f"Files generated: {len(sdf_files)}")
            else:
                self.log(f"Warning: Output folder not found: {output_path}")
            
            self.log("=" * 80)
            
            self.run_counter += 1
            self.total_runs += 1
            
            return True
            
        except Exception as e:
            self.log("=" * 80)
            self.log(f"Run #{self.run_counter} encountered an error")
            self.log(f"Error message: {e}")
            self.log("=" * 80)
            return False
    
    def run_continuous(self):
        self.log("Starting continuous running loop...")
        
        print(f"\nStarting continuous run, expected duration {self.duration_hours} hours")
        print(f"Run progress will be displayed via progress bar")
        
        pbar = tqdm(desc="Run progress", unit="runs", total=None, bar_format='{desc}: {n} runs | Running time: {elapsed} | Remaining: {postfix}')
        
        try:
            while self.is_time_remaining():
                remaining = self.format_time(self.get_remaining_time())
                pbar.set_postfix_str(remaining)
                
                success = self.run_delete_once()
                
                pbar.update(1)
                pbar.set_description(f"Completed {self.total_runs} runs")
                
                if not self.is_time_remaining():
                    self.log("Reached the set running duration, stopping")
                    break
                
                if WAIT_BETWEEN_RUNS > 0:
                    self.log(f"Waiting {WAIT_BETWEEN_RUNS} seconds before starting next run")
                    time.sleep(WAIT_BETWEEN_RUNS)
                
        except KeyboardInterrupt:
            self.log("=" * 80)
            self.log("User interrupted the run")
            self.log(f"Total runs completed: {self.total_runs}")
            self.log("=" * 80)
            print(f"\nUser interrupted the run")
        
        finally:
            pbar.close()
            
            actual_duration = (datetime.datetime.now() - self.start_time).total_seconds() / 3600
            
            self.log("=" * 80)
            self.log("Run ended")
            self.log(f"Total runs: {self.total_runs}")
            self.log(f"Actual running duration: {actual_duration:.2f} hours")
            self.log(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
            self.log("=" * 80)
            
            print(f"\n{'='*80}")
            print(f"Run complete!")
            print(f"Total runs: {self.total_runs}")
            print(f"Actual running duration: {actual_duration:.2f} hours")
            print(f"Detailed log: {self.log_file}")
            print(f"{'='*80}")

runner = DeleteAutoRunnerV2(
    delete_root=DELETE_ROOT,
    config=DELETE_CONFIG,
    duration_hours=RUN_DURATION_HOURS,
    use_cpu=USE_CPU_VERSION,
    log_file=LOG_FILE
)

runner.run_continuous()